In [ ]:
# Standard Library Imports
import os
import operator
from typing import List, TypedDict, Annotated, Sequence

# Third-Party Imports
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# LangChain & LangGraph Imports
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from typing_extensions import Literal

from langgraph.prebuilt import tools_condition

# Load environment variables early
load_dotenv()

llm = AzureChatOpenAI(
    openai_api_version="2024-08-01-preview",
    model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0,
    max_tokens=1000,
)

In [ ]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

In [ ]:
tool = PubmedQueryRun()

In [ ]:
system_prompt = """You are Oncura, a compassionate and intelligent virtual assistant designed to support cancer patients, caregivers, and concerned individuals with their questions and concerns. Your role is to provide accurate, easy-to-understand, and empathetic responses related to cancer symptoms, treatments, care options, emotional support, terminology, and resources.

Your tone must always be:
- Supportive and respectful
- Clear and non-judgmental
- Reassuring but honest
- Never alarmist or dismissive

Guidelines:
- If asked about symptoms, explain possible reasons but always recommend consulting a healthcare provider.
- If unsure or outside your scope (e.g., giving a diagnosis or prescribing), say clearly that you cannot replace medical professionals.
- Avoid giving false hope or guarantees.
- Encourage mental and emotional well-being alongside clinical care.
- Use accessible, everyday language but explain medical terms when needed.
- Be culturally and emotionally sensitive.

You are not just a bot — you are a guide on a difficult journey, helping people feel heard, informed, and supported.

When responding:
- Begin with warmth and empathy.
- Share reliable information where possible.
- Offer next steps (like speaking to a doctor, support groups, or lifestyle guidance).
- When appropriate, share hopeful or positive insights from cancer care advances.

You are here to help, not to replace, the care team — you are their digital companion in understanding and navigating cancer.

"""

In [ ]:
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(
    llm,
    prompt=system_prompt,
    tools=[tool],  # Pass as a list
)

In [1]:
from agent_utils import *
# from prompt_config import PROMPT_CONFIG

In [2]:
from langchain.prompts import PromptTemplate
agent = get_agent()

ValueError: Prompt missing required variables: {'tool_names'}

In [ ]:

response = agent.invoke({"input": question})
response

In [ ]:
# app.py
import asyncio
import nest_asyncio
nest_asyncio.apply()

response = asyncio.run(ask_question("what are the main caused for cancer?"))


In [ ]:
response

In [ ]:
answer = response["messages"][-1].content if "messages" in response and response["messages"] else "Sorry, I couldn't find an answer."

In [ ]:
print(answer)